## Training 

In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
import pickle
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
#Loading data and embeddings
label_embeddings = pickle.load(open("Data/Embeddings/CIFAR/CIFAR_100_label_to_embedding_google_news.pk", "rb"))
train_x = np.load("vgg_16/train_x.npy").reshape((50000,8192))
train_y = np.load("vgg_16/train_y.npy")
test_x = np.load("vgg_16/test_x.npy").reshape((10000,8192))
test_y = np.load("vgg_16/test_y.npy")


In [3]:
#Convert train/test labels to embeddings
train_y_embeddings = np.zeros((50000, 300))
test_y_embeddings = np.zeros((10000, 300))

for i in range(train_y.shape[0]):
    train_y_embeddings[i] = label_embeddings[int(train_y[i][0])]

for i in range(test_y.shape[0]):
    test_y_embeddings[i] = label_embeddings[int(test_y[i][0])]

In [4]:
model = Sequential()
model.add(Dense(1024, input_dim=8192, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, input_dim=300, activation='relu'))
model.add(BatchNormalization())
#model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dense(300, input_dim=300))

In [5]:
def cos_distance(y_true, y_pred):
    def l2_normalize(x, axis):
        norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=True))
        return K.sign(x) * K.maximum(K.abs(x), K.epsilon()) / K.maximum(norm, K.epsilon())
    y_true = l2_normalize(y_true, axis=-1)
    y_pred = l2_normalize(y_pred, axis=-1)
    return K.mean(y_true * y_pred, axis=-1)

In [6]:
model.compile(loss='mse', optimizer='adam', metrics=['cosine_proximity'])

In [7]:
model.fit(train_x, train_y_embeddings, epochs=50, batch_size=128, validation_data=(test_x, test_y_embeddings))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 7s 131us/step - loss: 0.0562 - cosine_proximity: -0.3576 - val_loss: 0.0321 - val_cosine_proximity: -0.4215
Epoch 2/50
50000/50000 [==============================] - 5s 102us/step - loss: 0.0306 - cosine_proximity: -0.4452 - val_loss: 0.0315 - val_cosine_proximity: -0.4358
Epoch 3/50
50000/50000 [==============================] - 5s 97us/step - loss: 0.0296 - cosine_proximity: -0.4671 - val_loss: 0.0304 - val_cosine_proximity: -0.4723
Epoch 4/50
50000/50000 [==============================] - 5s 97us/step - loss: 0.0292 - cosine_proximity: -0.4772 - val_loss: 0.0291 - val_cosine_proximity: -0.4787
Epoch 5/50
50000/50000 [==============================] - 5s 97us/step - loss: 0.0286 - cosine_proximity: -0.4897 - val_loss: 0.0294 - val_cosine_proximity: -0.4737
Epoch 6/50
50000/50000 [==============================] - 5s 97us/step - loss: 0.0281 - cosine_proximity: -0.4994 - val_los

50000/50000 [==============================] - 5s 97us/step - loss: 0.0188 - cosine_proximity: -0.6792 - val_loss: 0.0239 - val_cosine_proximity: -0.5988


## Testing

In [8]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [9]:
pred_test_y = model.predict(test_x)

label_embeddings
label_embeddings_arr = np.zeros((100, 300))
for i in range(100):
    label_embeddings_arr[i] = label_embeddings[i]

In [10]:
#Top K Predictions
def pred_top_k(k=5, pred_test_y=pred_test_y, label_embeddings_arr=label_embeddings_arr):
    sim_table = cosine_similarity(pred_test_y, label_embeddings_arr)
    top_k_guesses = np.argpartition(sim_table,range(99-k+1,100),axis=1)[:,99-k+1:]
    return top_k_guesses


#Top prediction
sim_table = cosine_similarity(pred_test_y, label_embeddings_arr)
label_predictions = sim_table.argmax(axis=1)
label_predictions

k=5
top_k_guesses = pred_top_k(5)


In [11]:
#Accuracy
acc = np.sum((np.squeeze(test_y) == label_predictions)) / float(test_y.shape[0])
print("Accuracy: " + str(acc))

#Top k Accuracy

def acc(top_k=top_k_guesses, test_y=test_y):
    correct = 0
    for i in range(test_y.shape[0]):
        if np.squeeze(test_y)[i] in top_k[i]:
            correct += 1
    return correct/float(test_y.shape[0])

print("Top " + str(k) + " Accuracy: " + str(acc()))

Accuracy: 0.3103
Top 5 Accuracy: 0.4837


In [12]:
x = ny = np.zeros(100)

p.linspace(0,99,100)
for i in range(100):
    k = i+1
    top_k_guesses = pred_top_k(k)
    y[i] = acc(top_k_guesses, test_y)

NameError: name 'p' is not defined

In [ ]:
plt.plot(x,y)
plt.title("Top K Accuracy")